# 减少内存位数

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

这段代码的目的是减少DataFrame的内存占用。它首先计算DataFrame的初始内存大小，并遍历所有列，检查它们的数据类型。如果列的数据类型是整数或浮点数，它将尝试将列转换为更小的数据类型，以减少内存使用。

代码逻辑如下：

1. 定义一个列表`numerics`，包含了所有需要检查的数值数据类型。
2. 计算DataFrame `df`的初始内存使用量`start_mem`。
3. 遍历DataFrame的所有列，对于每一列：
    - 获取列的数据类型`col_type`。
    - 如果`col_type`是数值类型之一：
        - 计算列的最小值`c_min`和最大值`c_max`。
        - 如果`col_type`是整数类型：
            - 根据`c_min`和`c_max`的值，尝试将列转换为`np.int8`、`np.int16`、`np.int32`或`np.int64`中的一个更小的整数类型。
        - 如果`col_type`是浮点类型：
            - 计算列的最大精度`c_prec`。
            - 根据`c_min`、`c_max`和`c_prec`的值，尝试将列转换为`np.float16`、`np.float32`或保留为`np.float64`中的一个更小的浮点类型。
4. 计算转换后的DataFrame `df`的内存使用量`end_mem`。
5. 如果`verbose`参数为True，打印内存使用量减少的信息，包括减少的百分比。

总的来说，这个函数通过转换DataFrame中的列到更小的数据类型来减少内存占用，特别是对于包含大量数值数据的大型DataFrame，这可以带来显著的内存节省。

# 编码拼接

In [ ]:
df = pd.concat([train_df,test_df],axis=0).reset_index(drop=True)

这行代码是Pandas库中的一段代码，它的功能是将两个DataFrame对象 `train_df` 和 `test_df` 沿着轴合并，并重新设置索引。

1. `pd.concat()` 函数是Pandas中用来合并两个或多个DataFrame或Series的函数。这里，它被用来合并 `train_df` 和 `test_df` 这两个DataFrame。

2. `axis=0` 参数指定了合并的轴。在这个例子中，`axis=0` 表示沿着行的方向（垂直方向）合并，即将 `test_df` 放在 `train_df` 的下方。如果设置为 `axis=1`，则会沿着列的方向（水平方向）合并，即将 `test_df` 放在 `train_df` 的右侧。

3. `reset_index()` 函数用来重置DataFrame的索引。在合并两个DataFrame之后，每个原始DataFrame的索引都会保留在结果DataFrame中，这可能会导致索引重复。为了避免这个问题，`reset_index()` 会重新为合并后的DataFrame生成一个新的连续索引。

4. `drop=True` 参数指定了在重置索引时，是否丢弃原来的索引。如果设置为 `True`，那么原来的索引会被丢弃，不会成为结果DataFrame的一列。如果设置为 `False`，原来的索引会被保留并作为一个新的列添加到结果DataFrame中。

综上所述，这行代码的整体作用是将 `train_df` 和 `test_df` 这两个DataFrame垂直方向合并成一个新的DataFrame，并为这个新的DataFrame生成一组新的连续索引，同时丢弃原来的

# 离散特征编码

In [ ]:
# 这段代码定义了两个列表：num_cols 和 cate_cols，它们分别用于存储数值型特征（numerical features）和分类特征（categorical features）的列名。
num_cols  =  [f'user_fea{i}' for i in range(6,9)] 
cate_cols = ['user_id','item_id']+ [f'user_fea{i}' for i in range(1,6)] + ['item_fea1']+['user_fea9']

In [ ]:
# 原生离散特征编码
for col in tqdm(cate_cols):
    map_dict = dict(zip(df[col].unique(), range(df[col].nunique())))
    # label enc
    df[col] = df[col].map(map_dict)
    # count enc
    df[f'{col}_count'] = df[col].map(df[col].value_counts())

这段代码遍历一个名为 `cate_cols` 的列表，该列表包含DataFrame `df` 中分类特征（categorical features）的列名。对于每个分类特征，代码执行以下操作：

1. 使用 `zip()` 函数和 `dict()` 函数创建一个映射字典 `map_dict`，将每个唯一值映射到一个整数。`df[col].unique()` 返回列中的所有唯一值，而 `range(df[col].nunique())` 生成一个与唯一值数量相同的连续整数序列。这个映射字典用于将分类特征的文本值转换为整数标签。

2. 使用 `map()` 函数，将 `map_dict` 应用到列 `df[col]` 上，实现标签编码（label encoding）。标签编码是将每个唯一的分类值转换为一个整数。

3. 创建一个新列，名为 `f'{col}_count'`，其中 `f'{col}_count'` 是一个格式化字符串，它会被替换为原始列名后加上 `_count`。例如，如果原始列名是 `category`，新列名将是 `category_count`。

4. 对于新创建的 `f'{col}_count'` 列，使用 `map()` 函数将 `df[col].value_counts()` 映射到 `df[col]` 上。`df[col].value_counts()` 返回一个Series，包含每个唯一值在原始列中出现的次数。这样，新列将包含原始分类值在整个列中的出现次数，实现计数编码（count encoding）。

整个代码块的目的是对DataFrame中的分类特征进行处理，将它们转换为数值形式，以便于机器学习模型的使用。标签编码和计数编码是处理分类数据的常见方法。

`tqdm` 是一个快速、可扩展的Python进度条库，它可以在长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 `tqdm(iterator)`。在这段代码中，它被用来包装 `cate_cols` 列表，以便于在遍历分类特征时显示进度条。

## 交叉特征

In [ ]:
item_cate_feature = ['item_fea1']
user_cate_feature = [f'user_fea{i}' for i in range(1,6)] + ['user_fea9']
cross_feature_pair=[
    ['user_id','item_id']
]
# cross_feature_pair += [['user_id',fea] for fea in item_cate_feature]
# cross_feature_pair += [['item_id',fea] for fea in user_cate_feature]

for [f1,f2] in tqdm(cross_feature_pair):
    df[f'cross_{f1}_{f2}'] = df[f1].astype('str') + '_' + df[f2].astype('str')
    
    df[f'cross_{f1}_{f2}_count'] = df[f'cross_{f1}_{f2}'].map(df[f'cross_{f1}_{f2}'].value_counts())
    
    df[f'cross_{f1}_{f2}/{f1}'] = df[f'cross_{f1}_{f2}_count'] / (df[f'{f1}_count']+df[f'{f1}_count'].mean())
    df[f'cross_{f1}_{f2}/{f2}'] = df[f'cross_{f1}_{f2}_count'] / (df[f'{f2}_count']+df[f'{f2}_count'].mean())
    
    del df[f'cross_{f1}_{f2}']
df = reduce_mem_usage(df)

这段代码用于创建交叉特征（cross features），并对这些特征进行计数编码和比率计算，以用于机器学习模型中。具体步骤如下：

1. 定义了两个列表 `item_cate_feature` 和 `user_cate_feature`，分别包含了物品和用户的分类特征。

2. 定义了一个列表 `cross_feature_pair`，它包含了需要交叉的特征对。目前，它只有一个元素：`['user_id', 'item_id']`。

3. 注释掉的两行代码是用来扩展 `cross_feature_pair` 列表的，它们将 `user_id` 与 `item_cate_feature` 中的每个特征进行交叉，以及将 `item_id` 与 `user_cate_feature` 中的每个特征进行交叉。这将创建更多的交叉特征对，但目前这部分代码是被注释掉的，所以不会执行。

4. 使用 `tqdm` 进行遍历，为每个交叉特征对创建新的特征：
   - 使用 `astype('str')` 将两个特征 `f1` 和 `f2` 转换为字符串，并通过 `'_'` 连接它们，创建一个新的交叉特征 `f'cross_{f1}_{f2}'`。
   
   - 使用 `value_counts()` 计算新创建的交叉特征的计数，并将其保存到新列 `f'cross_{f1}_{f2}_count'` 中。
   
   - 创建两个新的比率特征 `f'cross_{f1}_{f2}/{f1}'` 和 `f'cross_{f1}_{f2}/{f2}'`。这些特征分别是交叉特征计数除以 `f1` 和 `f2` 的计数加上它们各自计数的平均值。这可以帮助捕捉到 `f1` 和 `f2` 在交叉特征中相对于它们单独计数的重要性。
   
   - 最后，删除了原始的交叉特征列 `f'cross_{f1}_{f2}'`，因为它的信息已经被编码到了其他的特征中。

总之，这段代码的作用是创建基于分类特征组合的新特征，并计算这些特征的计数和比率。这些衍生特征可能会帮助模型更好地理解特征之间的交互关系，从而提高模型的性能。

## 全局统计特征

In [ ]:
global_cate_group_feature = [['user_id',fea] for fea in item_cate_feature]+[['item_id',fea] for fea in user_cate_feature]
# 离散特征-离散特征
for [key,value] in tqdm(global_cate_group_feature):
    tmp = df.groupby([key])[value].nunique().reset_index()
    tmp.columns = [key, value+'_nunique']
    df = df.merge(tmp, on=key, how='left')
    
#离散特征-连续特征
global_dense_group_feature =[['item_id',f'user_fea{i}']  for i in range(6,9) ]
for [key,value] in tqdm(global_dense_group_feature):
    tmp = df.groupby([key])[value].describe().reset_index()
    tmp.columns = [key] + ['group_'+ x for x in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
    df = df.merge(tmp[[key] + ['group_'+ x for x in ['mean', 'std', '25%', '50%', '75%']]], on=key, how='left')

df = reduce_mem_usage(df)

这段代码用于生成基于分组的统计特征（grouped statistics）并将这些特征合并到原始DataFrame `df` 中。代码分为两个部分，分别处理离散特征与离散特征之间的关系和离散特征与连续特征之间的关系。

### 离散特征-离散特征

1. 定义 `global_cate_group_feature` 列表，它包含了需要进行分组统计的离散特征对。列表由两部分组成：一部分是 `user_id` 与 `item_cate_feature` 中每个特征的组合，另一部分是 `item_id` 与 `user_cate_feature` 中每个特征的组合。

2. 遍历 `global_cate_group_feature` 列表中的每个特征对 `[key, value]`：
   - 使用 `groupby()` 和 `nunique()` 对 `key` 特征进行分组，并计算每个组中 `value` 特征的唯一值数量。
   - 重置索引并为返回的DataFrame设置新的列名，其中第二列的名字是 `value+'_nunique'`，表示每个组中唯一值的数量。
   - 使用 `merge()` 将上述统计结果合并回原始DataFrame `df`，根据 `key` 进行左连接。

### 离散特征-连续特征

1. 定义 `global_dense_group_feature` 列表，它包含了需要进行分组统计的离散特征与连续特征对。列表由 `item_id` 与 `user_fea6`、`user_fea7` 和 `user_fea8` 的组合构成。

2. 遍历 `global_dense_group_feature` 列表中的每个特征对 `[key, value]`：
   - 使用 `groupby()` 和 `describe()` 对 `key` 特征进行分组，并获取每个组中 `value` 特征的描述性统计信息（如计数、均值、标准差、最小值、四分位数等）。
   - 重置索引并为返回的DataFrame设置新的列名，列名前缀为 `group_`，后接描述性统计信息的名称。
   - 使用 `merge()` 将上述统计结果的子集（只包括均值、标准差和四分位数）合并回原始DataFrame `df`，根据 `key` 进行左连接。

总的来说，这段代码的作用是为原始DataFrame `df` 创建额外的统计特征，这些特征可能有助于模型捕捉到数据中的某些模式或关系，从而提高模型的性能。在机器学习的特征工程阶段，这类统计特征经常被用来增强模型的表现力。

## 分割训练验证集

In [ ]:
train_df = df[df['label'].notna()].reset_index(drop=True)
test_df = df[df['label'].isna()].reset_index(drop=True)

del df
gc.collect()

这段代码的作用是将包含训练数据和测试数据的DataFrame `df` 分割为两个独立的DataFrame：`train_df` 用于训练模型，`test_df` 用于模型的测试或预测。

1. `train_df = df[df['label'].notna()].reset_index(drop=True)`
   - `df['label'].notna()` 返回一个布尔序列，其中值为 `True` 的元素表示 `label` 列的值不是 `NaN`（即非空值），这通常意味着这些行是训练数据。
   - `df[df['label'].notna()]` 通过布尔索引选择了所有 `label` 列非空的行，即选择了训练集。
   - `reset_index(drop=True)` 重置DataFrame的索引，使其从0开始并连续，`drop=True` 参数表示不保留原索引。

2. `test_df = df[df['label'].isna()].reset_index(drop=True)`
   - `df['label'].isna()` 返回一个布尔序列，其中值为 `True` 的元素表示 `label` 列的值是 `NaN`（即空值），这通常意味着这些行是测试数据。
   - `df[df['label'].isna()]` 通过布尔索引选择了所有 `label` 列为空的行，即选择了测试集。
   - 同样使用 `reset_index(drop=True)` 重置索引。

3. `del df`
   - 删除原始的DataFrame `df` 以释放内存。

4. `gc.collect()`
   - 调用Python的垃圾回收器来进一步清理释放内存。在删除了大型对象或多个数据结构之后，执行垃圾回收可以帮助回收未被引用的内存。这在处理大型数据集时尤其有用，可以减少内存占用并防止程序崩溃。

总之，这段代码是在数据预处理阶段使用的，用于将数据集分割为训练集和测试集，并进行内存管理以确保数据处理过程中的效率。

## 交叉验证

StratifiedKFold 是来自Python的scikit-learn库中的一个类，它用于分层交叉验证。交叉验证是一种评估模型性能的技术，将数据集分成训练集和验证集的多个不同的组合，并在这些组合上训练和评估模型。分层交叉验证保证每个折（fold）中各个类别的样本比例与整个数据集中的比例一致。

以下是 StratifiedKFold 的一些关键特点：

1.分层采样：在分类问题中，特别是当数据集的目标变量的分布不均匀时，StratifiedKFold 会尝试在每个折中保持每个类别的比例与完整数据集中的比例相同。这是通过在分割数据前进行分层采样来实现的。

2.k折划分：n_splits 参数定义了数据集将被划分成多少个折。例如，n_splits=5 表示数据集将被划分为5个折。

3.随机化：shuffle 参数控制是否在划分前随机打乱数据。random_state 参数用于指定随机数生成器的种子，以确保结果的可重复性。

4.迭代器：StratifiedKFold 是一个迭代器，可以生成一系列的训练集和验证集的索引。这些索引可以用来创建实际的数据集划分。

In [ ]:
## 五折目标编码
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for f in tqdm(cate_cols):
    train_df[f + '_target_enc'] = 0
    test_df[f + '_target_enc'] = 0
    for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
        trn_x = train_df[[f, 'label']].iloc[trn_idx].reset_index(drop=True)
        val_x = train_df[[f]].iloc[val_idx].reset_index(drop=True)
        
        enc_df = trn_x.groupby(f, as_index=False)['label'].agg({f + '_target_enc': 'mean'})
        
        val_x = val_x.merge(enc_df, on=f, how='left')
        test_x = test_df[[f]].merge(enc_df, on=f, how='left')
        
        val_x[f + '_target_enc'] = val_x[f + '_target_enc'].fillna(train_df['label'].mean())
        test_x[f + '_target_enc'] = test_x[f + '_target_enc'].fillna(train_df['label'].mean())
        # 前面的可以类比做交叉验证的模型训练
        train_df.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values # 相当于oof_pre的赋值
        test_df[f + '_target_enc'] += test_x[f + '_target_enc'].values / skf.n_splits # 相当于test_df的均值操作

train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

这段代码执行了五折分层交叉验证的目标编码（也称为均值编码或类别编码）。目标编码是一种特征工程技术，用于将分类变量转换为与目标变量相关的数值型特征。在这种情况下，目标变量是 `label` 列。

代码的步骤如下：

1. 创建一个 `StratifiedKFold` 对象 `skf`，它将数据集分成5个部分，确保每个部分的标签分布与原始数据集的分布相似。`shuffle=True` 表示在分割之前将数据随机打乱，`random_state=2020` 用于确保结果的可重复性。

2. 遍历 `cate_cols` 列表中的每个分类特征 `f`：
   - 初始化训练集和测试集中该特征的目标编码列（`f + '_target_enc'`）为0。

3. 对于每个特征 `f`，使用 `skf.split()` 方法进行五折分层交叉验证：
   - 在每一折中，将训练数据分为训练部分 `trn_x` 和验证部分 `val_x`。
   - 使用训练部分的数据计算 `f` 特征的每个类别的目标平均值，即每个类别对应的 `label` 的均值。这些均值保存在 `enc_df` DataFrame中。
   - 将 `enc_df` 中的目标编码均值与验证部分 `val_x` 和测试集 `test_x` 进行合并，以便于训练集的验证部分和测试集都能使用训练部分计算得到的目标编码。
   - 使用训练集的 `label` 均值填充验证部分和测试集中的缺失值。这是为了处理那些在训练部分中没有出现的类别。
   - 将验证部分的目标编码值赋给原始训练集的对应行（这类似于预测验证部分的标签）。
   - 将测试集的目标编码值累加，并最终取平均（通过除以折数 `skf.n_splits`）。

4. 最后，调用先前定义的 `reduce_mem_usage()` 函数来减少训练集和测试集的内存占用。

总体而言，这段代码通过五折分层交叉验证的方式为每个分类特征创建了一个基于目标变量的均值编码特征。这种方法可以减少目标泄露（target leakage）的风险，因为每次都是使用不包含验证部分数据的训练部分来计算均值。目标编码特征通常可以为模型提供有关类别与目标之间关系的有用信息，从而可能提高模型的性能。然而，需要注意防止过拟合，特别是当某些类别的样本量很小的时候。